## Simulate data CRDM

This script is used to generate simulated data for the CRDM task. Then we can run parameter_recovery to estimate those parameters from the simulated data


In [49]:
# Built-in/Generic Imports
import os

# Libs
import numpy as np
import pandas as pd
from scipy.stats import bernoulli

In [50]:

### Defining the grids, and functions to make sets as dataframes: choice_set,param_set

def set_grids(nb_alpha=10,nb_beta=10,nb_gamma=10):
    grid_design = {
        # probability of safe bet
        'crdm_sure_p': [1.0],
        # probability of reward (lottery winning probability)
        'crdm_lott_p': [0.13, 0.25, 0.38, 0.50, 0.75],
        # safe reward: $5
        'crdm_sure_amt': [5],
        # reward amount set to vary according to experiment
        'crdm_lott_amt': [8, 10, 12, 15, 17, 20, 23, 26, 30, 33, 40, 43,47, 50],
        # ambiguity levels
        'crdm_amb_lev':[0.0, 0.24, 0.50, 0.74]
    }
    grid_param = {
        'alpha_gt': np.logspace(-2, 0.5, nb_alpha+1,base=np.exp(1))[1:],
        # 'beta': np.linspace(-3, 3, 11),
        # setting beta_max = 1.3 to avoid getting negative SV when generating new choice set
        'beta_gt': np.linspace(-2, 2, nb_beta),
        'gamma_gt': np.linspace(0, 8, nb_gamma+1)[1:]
    }
    return grid_design,grid_param

def insert_var(grid=[],var_nb=0,list_var=[]):
    if not var_nb:
        grid = list_var
    elif var_nb==1:
        grid = [[g]+[i] for g in grid for i in list_var]
    else:
        grid = [g+[i] for g in grid for i in list_var]
    return grid

def make_grid(grid_values):
    grid = []
    labels = grid_values.keys()
    for var_nb,row in enumerate(grid_values.items()):
        grid = insert_var(grid=grid,var_nb=var_nb,list_var=row[1])
    grid_df = pd.DataFrame(grid,columns=labels)
    return grid_df

def remove_ambig_trials(df):
    # find trials that we want to remove: 50% lottery probability and > 0 ambiguity
    # then take the not of that to keep the remaining trials
    df = df.loc[~((df['crdm_lott_p']!=0.5)&(df['crdm_amb_lev']>0))].reset_index(drop=True)
    return df

def append_lott_top_bot(df):
    df['crdm_lott_top'] = df['crdm_lott_amt']
    df['crdm_lott_bot'] = 0
    return df

In [51]:

def simulate_response(design,PARAM_TRUE):
    # Calculate the probability to choose a variable option
    pn, pr, vn, vr, ambig = (design['crdm_sure_p'], design['crdm_lott_p'],design['crdm_sure_amt'], design['crdm_lott_amt'],design['crdm_amb_lev'])
    alpha, beta, gamma = PARAM_TRUE['alpha_gt'], PARAM_TRUE['beta_gt'], PARAM_TRUE['gamma_gt']

    SV_null = (vn**alpha) * pn
    SV_reward = (vr**alpha) * (pr - beta * ambig / 2)
    p_obs = 1. / (1 + np.exp(-gamma * (SV_reward - SV_null)))
    # debugging
    # print('(alpha,beta,gamma,SV_difference) : ({0:0.3f},{1:0.3f},{2:0.3f},{3:0.3f})'.format(alpha,beta,gamma,SV_reward-SV_null))


    # Randomly sample a binary choice response from Bernoulli distribution
    return float(bernoulli.rvs(p_obs))

def generate_response(design_df,PARAM_TRUE):
    response= []
    df = pd.DataFrame([])
    for index, design in design_df.iterrows():
        response += [simulate_response(design,PARAM_TRUE)]
    df['crdm_choice'] = response
    df['crdm_trial_type'] = 'task'
    df['crdm_domain'] = 'gain'
    df['crdm_confkey'] = 1.0
    return df


In [52]:
# can set resolution (number of parameter) for kappa and gamma
grid_design,grid_param = set_grids(nb_alpha=5,nb_beta=5,nb_gamma=5)

design_df = make_grid(grid_design)
# some trials have ambiguity > 0 and prob not 0.50
design_df = remove_ambig_trials(design_df)
design_df = append_lott_top_bot(design_df)
fn = os.path.join('simul','design_set.csv')
print('Saving design to : {}'.format(fn))
design_df.to_csv(fn)

param_df = make_grid(grid_param)
fn = os.path.join('simul','ground_truth.csv')
print('Saving ground truth parameters to : {}'.format(fn))
param_df.to_csv(fn)

for index, PARAM_TRUE in param_df.iterrows():
    response_df = generate_response(design_df,PARAM_TRUE)
    subj_dir = os.path.join('simul','split','p{0:04d}'.format(index),'crdm')
    if not os.path.exists(subj_dir):
        os.makedirs(subj_dir)
    fn = os.path.join(subj_dir,'p{0:04d}_crdm.csv'.format(index))
    print('Saving response to : {}'.format(fn))
    df = pd.concat([design_df,response_df],axis=1)
    df.to_csv(fn)


Saving design to : simul/design_set.csv
Saving ground truth parameters to : simul/ground_truth.csv
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.225)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.215)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.206)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.194)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.187)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.178)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.170)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.163)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.154)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.148)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.136)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.131)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.125)
(alpha,beta,gamma,SV_difference) : (0.223,-2.000,1.600,-1.121)
(alpha,beta,gamma,S

/var/folders/ts/wpzrly5j2yxb42zf5v0w5rvh0000gs/T/ipykernel_32675/1832438162.py:8: RuntimeWarning: overflow encountered in exp
  p_obs = 1. / (1 + np.exp(-gamma * (SV_reward - SV_null)))
/var/folders/ts/wpzrly5j2yxb42zf5v0w5rvh0000gs/T/ipykernel_32675/1832438162.py:8: RuntimeWarning: overflow encountered in exp
  p_obs = 1. / (1 + np.exp(-gamma * (SV_reward - SV_null)))
/var/folders/ts/wpzrly5j2yxb42zf5v0w5rvh0000gs/T/ipykernel_32675/1832438162.py:8: RuntimeWarning: overflow encountered in exp
  p_obs = 1. / (1 + np.exp(-gamma * (SV_reward - SV_null)))
